In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

# This is for testing
# df = spark.read \
#     .option("header", "true") \
#     .csv('taxi_zone_lookup.csv')

# df.show()
# df.write.parquet('zones', mode='overwrite')

25/01/11 23:19:46 WARN Utils: Your hostname, mackutis.local resolves to a loopback address: 127.0.0.1; using 192.168.1.147 instead (on interface en0)
25/01/11 23:19:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/11 23:19:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# # download the data from github 
# import requests 

# url = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz"
local_path = "fhvhv_tripdata_2021-01.csv.gz"

# response = requests.get(url)
# response.raise_for_status()

# with open(local_path, 'wb') as f:
#     f.write(response.content)

In [5]:
# !gzip -dc fhvhv_tripdata_2021-01.csv.gz > fhvhv_tripdata_2021-01.csv

In [6]:
!head -n 10 fhvhv_tripdata_2021-01.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,


In [7]:
# enforce schema
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

schema = StructType([
    StructField("hvfhs_license_num", StringType(), True),
    StructField("dispatching_base_num", StringType(), True),
    StructField("pickup_datetime", TimestampType(), True),
    StructField("dropoff_datetime", TimestampType(), True),
    StructField("PULocationID", IntegerType(), True),
    StructField("DOLocationID", IntegerType(), True),
    StructField("SR_Flag", StringType(), True),
])

In [12]:
# Load the data
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv(local_path)

In [13]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   NULL|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   NULL|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   NULL|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   NULL|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   NULL|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [14]:
# Parallel partitioning
df = df.repartition(24)

df.write.parquet('fhvhv/2021/01', mode='overwrite')

25/01/11 23:22:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/11 23:22:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/11 23:22:18 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/11 23:22:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/01/11 23:22:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [15]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [17]:
(
    df.select('hvfhs_license_num', 'dispatching_base_num', 'pickup_datetime', 'dropoff_datetime')
    .filter(df.hvfhs_license_num == 'HV0003')
    .show()
)


+-----------------+--------------------+-------------------+-------------------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|
+-----------------+--------------------+-------------------+-------------------+
|           HV0003|              B02764|2021-01-13 16:06:51|2021-01-13 16:17:53|
|           HV0003|              B02682|2021-01-10 05:04:16|2021-01-10 05:11:36|
|           HV0003|              B02871|2021-01-20 07:41:15|2021-01-20 08:08:25|
|           HV0003|              B02882|2021-01-07 06:48:41|2021-01-07 06:54:43|
|           HV0003|              B02888|2021-01-12 16:26:32|2021-01-12 16:32:20|
|           HV0003|              B02872|2021-01-30 21:07:43|2021-01-30 21:23:48|
|           HV0003|              B02765|2021-01-15 15:41:07|2021-01-15 15:46:48|
|           HV0003|              B02876|2021-01-07 15:38:48|2021-01-07 16:41:35|
|           HV0003|              B02871|2021-01-23 07:51:07|2021-01-23 08:12:27|
|           HV0003|         